# **Installations**

# **Calling Alma Database**

In [1]:
import numpy as np
from astropy.table import Table
import pyvo
import sys
import matplotlib.pyplot as plt
import pandas as pd
import sklearn.cluster

service = pyvo.dal.TAPService("https://almascience.eso.org/tap")      # for the EU ALMA TAP service

# service = pyvo.dal.TAPService("https://almascience.nao.ac.jp/tap")  # for the EA ALMA TAP service
# service = pyvo.dal.TAPService("https://almascience.nrao.edu/tap")   # for the NA ALMA TAP service

# **Navigation through Alma Database Schema**

In [2]:
def list_table_columns(service, table_name):
    """List all column names for a given table in the database.

    Parameters:
    service (pyvo.dal.TAPService): A TAPService instance for querying the database.
    table_name (str): The name of the table to list columns for.

    Returns:
    list: A list of column names in the specified table.
    """
    query = f"""
        SELECT column_name
        FROM TAP_SCHEMA.columns
        WHERE table_name = '{table_name}'
    """

    try:
        result = service.search(query).to_table()
        return result['column_name'].tolist()
    except Exception as e:
        print(f"An error occurred: {e}")
        return []

# Example usage:
service = pyvo.dal.TAPService("https://almascience.eso.org/tap")
column_names = list_table_columns(service, 'ivoa.obscore')
print(column_names)

['access_estsize', 'access_format', 'access_url', 'antenna_arrays', 'asdm_uid', 'authors', 'band_list', 'bandwidth', 'bib_reference', 'calib_level', 'collections', 'cont_sensitivity_bandwidth', 'data_rights', 'dataproduct_type', 'em_max', 'em_min', 'em_res_power', 'em_resolution', 'em_xel', 'facility_name', 'first_author', 'frequency', 'frequency_support', 'gal_latitude', 'gal_longitude', 'group_ous_uid', 'instrument_name', 'is_mosaic', 'lastModified', 'member_ous_uid', 'o_ucd', 'obs_collection', 'obs_creator_name', 'obs_id', 'obs_publisher_did', 'obs_release_date', 'obs_title', 'pi_name', 'pi_userid', 'pol_states', 'pol_xel', 'proposal_abstract', 'proposal_authors', 'proposal_id', 'pub_abstract', 'pub_title', 'publication_year', 'pwv', 'qa2_passed', 's_dec', 's_fov', 's_ra', 's_region', 's_resolution', 's_xel1', 's_xel2', 'scan_intent', 'schedblock_name', 'science_keyword', 'science_observation', 'scientific_category', 'sensitivity_10kms', 'spatial_resolution', 'spatial_scale_max', 't

# **Extracting Metadata from the Alma Database**

In [30]:
import pandas as pd
import pyvo

def query_observations(service, member_ous_uid, target_name):
    """Query for all science observations of given member OUS UID and target name, selecting all columns of interest.

    Parameters:
    service (pyvo.dal.TAPService): A TAPService instance for querying the database.
    member_ous_uid (str): The unique identifier for the member OUS to filter observations by.
    target_name (str): The target name to filter observations by.

    Returns:
    pandas.DataFrame: A table of query results.
    """

    query = f"""
            SELECT *
            FROM ivoa.obscore
            WHERE member_ous_uid = '{member_ous_uid}'
            AND target_name = '{target_name}'
            AND is_mosaic = 'F'
            """

    result = service.search(query).to_table().to_pandas()

    return result

def query_all_targets(service, targets):
    """Query observations for all predefined targets and compile the results into a single DataFrame.

    Parameters:
    service (pyvo.dal.TAPService): A TAPService instance for querying the database.
    targets (list of tuples): A list where each tuple contains (target_name, member_ous_uid).

    Returns:
    pandas.DataFrame: A DataFrame containing the results for all queried targets.
    """
    results = []

    for target_name, member_ous_uid in targets:
        result = query_observations(service, member_ous_uid, target_name)
        results.append(result)

    # Concatenate all DataFrames into a single DataFrame
    df = pd.concat(results, ignore_index=True)

    return df

# Predefined list of targets and their corresponding member OUS UIDs
targets = [
    ("J0842+1218", "uid://A001/X2fb/X3ec"),
    ("J0842+1218", "uid://A001/X885/X36b"),
    ("J0842+1218", "uid://A001/X1590/X397"),
    ("J0842+1218C1", "uid://A001/X1590/X1ef6"),
    ("SDSS_J092303.53+024739.5", "uid://A001/X12e/X2e6"),
    ("J1319+0950", "uid://A002/X391d0b/X1e"),
    ("J1319+0950", "uid://A002/X7fb989/X18"),
    ("J1319+0950", "uid://A001/X1465/X380c"),
    ("J1319+0950C1", "uid://A001/X1590/X1f06"),
    ("J1319+0950", "uid://A001/X2d20/X15d5"),
    ("ULASJ1319+0950", "uid://A001/X3621/X4412"),
    ("SDSS_J132853.66-022441.6", "uid://A001/X12e/X2ea"),
    ("PJ065-26", "uid://A001/X2fb/X3e4"),
    ("PJ065-26", "uid://A001/X1273/X36c"),
    ("PJ065-26", "uid://A001/X1465/X3830"),
    ("PSO_J167.6415-13.4960", "uid://A001/X2d6/X7d"),
    ("PJ231-20", "uid://A001/X2fb/X440"),
    ("PJ231-20", "uid://A001/X1465/X3810"),
    ("PJ231-20", "uid://A001/X1590/X3a3"),
    ("PJ231-20", "uid://A001/X15aa/X1f7"),
    ("PJ231-20", "uid://A001/X2d1f/X82d"),
    ("PJ231-20", "uid://A001/X2d1f/X829"),
    ("PJ308-21", "uid://A001/X2fb/X418"),
    ("PJ308-21", "uid://A001/X11a4/Xf"),
    ("PJ308-21", "uid://A001/X1465/X382c"),
    ("PJ308-21", "uid://A001/X1590/X38f"),
    ("PJ308-21", "uid://A001/X2d20/X3b14"),
    ("PJ308-21", "uid://A001/X2d20/X3b10"),
    ("WMH5", "uid://A001/X121/X24b"),
    ("WMH13", "uid://A001/X121/X24f"),
    ("WMH_5", "uid://A001/X2d6/X5a"),
    ("J1509-1749", "uid://A001/X2fb/X414"),
    ("J1509-1749", "uid://A001/X1465/X3824"),
    ("J1509-1749", "uid://A001/X1590/X11e2"),
    ("J1509-1749", "uid://A001/X1590/X11c6"),
    ("J1509-1749", "uid://A001/X1590/X11de"),
    ("J1306+0356", "uid://A001/X2fb/X40c"),
    ("J1306+0356", "uid://A001/X1273/X374"),
    ("J1306+0356", "uid://A001/X1590/X39f"),
    ("J1306+0356C1", "uid://A001/X1590/X1f02")
]


In [47]:
# Create a TAPService instance (replace 'your_service_url' with the actual URL)
service = pyvo.dal.TAPService("https://almascience.eso.org/tap")

# Query all targets and compile the results
df = query_all_targets(service, targets)

# **Variable Info**

# **Cleaning Variable Names**

In [48]:
df = df.drop_duplicates(subset='member_ous_uid')

In [49]:
# Define a dictionary to map existing column names to new names with unit initials
rename_columns = {
    'target_name': 'ALMA_source_name',
    'pwv': 'PWV',
    'schedblock_name': 'SB_name',
    'velocity_resolution': 'Vel.res',
    'spatial_resolution': 'Ang.res',
    's_ra': 'RA',
    's_dec': 'Dec',
    's_fov': 'FOV',
    't_resolution': 'Int.Time',
    't_max': 'Total.Time',
    'cont_sensitivity_bandwidth': 'Cont_sens_mJybeam',
    'sensitivity_10kms': 'Line_sens_10kms_mJybeam',
    'obs_release_date': 'Obs.date'
    
}

# Rename the columns in the DataFrame
df.rename(columns=rename_columns, inplace=True)

In [50]:
database = df[['ALMA_source_name', 'PWV', 'SB_name', 'Vel.res', 'Ang.res', 'RA', 'Dec', 'FOV', 'Int.Time', 'Total.Time', 'Cont_sens_mJybeam', 'Line_sens_10kms_mJybeam', 'Obs.date']]
database['Obs.date'] = database['Obs.date'].apply(lambda x: x.split('T')[0])


/tmp/ipykernel_3731516/3578391840.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  database['Obs.date'] = database['Obs.date'].apply(lambda x: x.split('T')[0])


OSError: Cannot save file into a non-existent directory: '/home/almasim/Documents/GitHub/ALMASim/metadata'

In [51]:
database.to_csv('/home/astro/Documents/GitHub/ALMASim/metadata/QSO_metadata.csv', index=False)

In [41]:
dates = database['Obs.date'].values
date = dates[0]
check = date.split('T')[0]
check

'2017-03-14'